# Computation of synthetic seismograms

Synthetic seismograms are obtained by running <b>computeSyntheticSeismograms</b> using the Green frequency wavenumber spectra computed before. Necessary prerequisites for doing so are an event file and a station file as well as event and station filters.

You should already have a working folder where you put the output from the Green FK spectra calculations. Please copy this notebook there, then close and halt this one, switch to your working folder and start it from there.

Since running programs takes very little time, we execute them from within this script. For this reason, we first need to set a path to the binaries:

In [1]:
import os
import sys
import subprocess
sys.path.append('/home/wolle/work_git/geminiUnified/python/')

In [2]:
binary_folder = '/home/wolle/work_git/geminiUnified/bin/'

## Event file

The following shows an example of an event file:

The first line specifies the coordinate system used to specify event locations. 'S' stands for spherical and 'C' for cartesian.<br> 
The next (wrapped) lines specify the events. Data fields provide: 
+ event ID, 
+ date and time strings containing year, month, day, hour, minute, second and nanoseconds of the event origin time, 
+ latitude in degrees, longitude in degrees, depth in km, 
+ magnitude, source type (0 = force, 1 = moment tensor), 
+ either 6 components of the moment tensor in the order $M_{rr}, M_{\theta\theta}, M_{\phi\phi},M_{r\theta},M_{r\phi},M_{\theta\phi}$, 
+ or three components of the force in the order $f_r, f_\theta, f_\phi$.

Use your text editor to create such a file!

## Station file

The following shows an example of a station file:

Again the first line specifies the coordinate system. The following lines specify the stations: 
+ station name, network name, 
+ latitude in degrees, longitude in degress and altitude in meters.

Use your text editor to create such a file.

## Setup of the parameter file

Running <b>computeSyntheticSeismograms</b> requires setting up a parameter file. We go through every parameter here step by step.

Base name for the output file of <b>computeGreenFKSpectraForSynthetics</b> with Green spectra in the
frequency-wavenumber domain. Provide full path and first part of file name including source type specification which is either <i>"FORCE"</i> for a single force or <i>"MOMENT"</i> for a moment tensor source. Further extensions such as <i>"000"</i> are provided automatically by <b>computeSyntheticSeismograms</b>.

In [18]:
basename = 'gfk.MOMENT'

Path to folder where synthetic seismograms are going to be stored (include "/" into the path).

In [19]:
path_synthetic_seis = './synseis/'

if (not os.path.exists(path_synthetic_seis)):
    os_mkdir(path_synthetic_seis)

Name of event file including full path.

In [20]:
file_event_list = './events'

Name of station list including full path.

In [21]:
file_station_list = './stations'

We also need event filters which are explained later. Here, we need to give the path to folder where event filters are stored (include "/" into the path). Filter file names follow the convention <i>"filter_eventid"</i>.

In [22]:
path_event_filter = './event_filter/'

if (not os.path.exists(path_event_filter)):
    os.mkdir(path_event_filter)

We also need station filters as explained later. Here, we need to gve the path to folder where station filters are stored (inlude / into the path). Filter file names follow the convention <i>"filter_staname_component"</i>.

In [23]:
path_station_filter = './station_filter/'

if (not os.path.exists(path_station_filter)):
    os.mkdir(path_station_filter)

Components to be evaluated are given a a character string composed of single characters each of whih specifies one component. Possible (implemented) values are:<br>
Z: vertical; N: north; E: east; H: water ressure (for hydrophones); R: $rr$-stress; S: $r\theta$-stress

In [24]:
components = 'ZNE'

Information specifying the way the event filter is computed.<br>
EVENT_FILTER_TYPE contains a keyword for the algorithm. Allowed values are currently: BUTTERWORTH_BANDPASS.<br>
EVENT_FILTER_SPECS contains a real array with necessary data.
Here, it is order and corner frequency (Hz) for highpass and lowpass filters,
respectively.

In [25]:
event_filter_type = 'BUTTERWORTH_BANDPASS'
event_filter_specs = '2.0 0.01 5.0 0.1'

Information specifying the way station filters are computed.<br>
STATION_FILTER_TYPE contains a keyword for the algorithm. Allowed values are currently "UNIT" (setting the filter value to 1 for all frequencies). More should be implemented later.

In [26]:
station_filter_type = 'UNIT'

Now, we are ready to generate the parameter file by importing the <b>parfile_synseis</b> parameter file template and running it with the variables defined above.

In [27]:
from parfile_synseis import create_synseis_parfile 

parstring = create_synseis_parfile(basename,path_synthetic_seis,file_event_list,file_station_list,
                                   path_event_filter,path_station_filter,components,event_filter_type,
                                   event_filter_specs,station_filter_type)

Now, we write the new parameter file into the compute folder. First, tell what its name is:

In [28]:
parfile = 'parfile_syn'

In [29]:
with open(parfile,'w') as fd:
    fd.write(parstring)

## Event filters

For each event, you may specify the transfer function of an event filter. It could represent the Fourier transform of a source wavelet or simply a filter to be applied to all seismograms generated by this event. The file contains one complex number per line. It is assumed that these numbers represent the values of the transfer function at the frequencies previously used by GEMINI for computing Green frequency-wavenumber spectra. Since producing this file manually is error prone, we offer the program <b>computeEventFilter</b> to do the job. <b>computeEventFilter</b> also used the previously constructed parameter file for input. Here is its usage message:

Run the <b>computeEventFilter</b> program in the compute folder.

In [33]:
message = subprocess.check_output(binary_folder+"computeEventFilter "+parfile,shell = True)
print(message)

b"--------------------------------------------------------------------------------\n                         | DOCUMENTATION OF COMMANDLINE CONTENTS |\n                         -----------------------------------------\nProgram:     computeEventFilter\nDescription: Compute event filters at frequencies specified in Green FK spectra for all events\n \nPositional arguments:\n         parfile_synseis: Type =  sval, Value = parfile_syn\n \nOptional arguments:\n--------------------------------------------------------------------------------\n The following values for the input parameters are used: \n 'FILE_EVENT_LIST' = './events'\n 'PATH_EVENT_FILTER' = './event_filter/'\n 'GFDSV_BASENAME' = 'gfk.MOMENT'\n 'EVENT_FILTER_TYPE' = 'BUTTERWORTH_BANDPASS'\n 'EVENT_FILTER_SPECS' = '3.0 0.01 5.0 0.1'\n >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n >>>>>>>>>>>>>>>>   SUCCESS   >>>>>>>>>>>>>>>\n >>>> \n >>>> SUCCESS in readStandardEventFile --> successfully opened event file './events'\n >>>> SUCCE

## Station filters

You may specify a filter transfer function for each station and component. Its main purpose is to provide the transfer function of the instrument. The filter files again consist of one column filled with complex numbers. It is assumed that these numbers represent the values of the transfer function at the frequencies previously used by GFDSV for computing Green frequency-wavenumber spectra. Since producing these files manually is error prone, we offer the program <b>computeStationFilter</b> to do the job. <b>computeStationFilter</b> also used the gfdsvseis parameter file for input. Here is its usage message:

Currently, this program does not support instrument transfer function calculations. It only offers a "unit" station filter actually doing nothing. But the output files are needed for <b>computeSyntheticSeismograms</b> to function correctly.

Run the <b>computeStationFilter</b> program in the compute folder.

In [36]:
message = subprocess.check_output(binary_folder+"computeStationFilter "+parfile,shell = True)
print(message)

b"--------------------------------------------------------------------------------\n                         | DOCUMENTATION OF COMMANDLINE CONTENTS |\n                         -----------------------------------------\nProgram:     computeStationFilter\nDescription: Compute station filters at frequencies specified in Green FK spectra for all events\n \nPositional arguments:\n         parfile_synseis: Type =  sval, Value = parfile_syn\n \nOptional arguments:\n--------------------------------------------------------------------------------\n The following values for the input parameters are used: \n 'FILE_STATION_LIST' = './stations'\n 'PATH_STATION_FILTER' = './station_filter/'\n 'GFDSV_BASENAME' = 'gfk.MOMENT'\n 'COMPONENTS' = 'ZNE'\n 'STATION_FILTER_TYPE' = 'UNIT'\n >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n >>>>>>>>>>>>>>>>   SUCCESS   >>>>>>>>>>>>>>>\n >>>> \n >>>> SUCCESS in readStandardStationFile --> successfully opened station file './stations'\n >>>> SUCCESS in readStandar

## Synthetic seismograms

Now, you are ready to run <b>computeSyntheticSeismograms</b> in the compute folder to which the parameter file was written. Here is its usage message:

In [37]:
message = subprocess.check_output(binary_folder+"computeSyntheticSeismograms "+parfile,shell = True)
print(message)

b"--------------------------------------------------------------------------------\n                         | DOCUMENTATION OF COMMANDLINE CONTENTS |\n                         -----------------------------------------\nProgram:     computeSyntheticSeismograms\nDescription: Compute synthetic seismograms using Green FK-spectra for synthetics\n \nPositional arguments:\n                 parfile: Type =  sval, Value = parfile_syn\n \nOptional arguments:\n--------------------------------------------------------------------------------\n The following values for the input parameters are used: \n 'GFDSV_BASENAME' = 'gfk.MOMENT'\n 'PATH_SYNTHETIC_SEIS' = './synseis/'\n 'FILE_EVENT_LIST' = './events'\n 'FILE_STATION_LIST' = './stations'\n 'COMPONENTS' = 'ZNE'\n 'PATH_EVENT_FILTER' = './event_filter/'\n 'PATH_STATION_FILTER' = './station_filter/'\n Desired depth of gfk-evaluation is:    15.0000000      km\n Index of gfk-evaluation is:           16\n True depth of gfk-evaluation is:       15.0000

Next, you should check the Green wavenumber spectra for proper selection of slowness limits. Cut-off of high amplitude branches can lead to inaccurate seismograms.

In [38]:
message = subprocess.check_output(binary_folder+"plotGreenFKSpectra -jr 15 "+basename,shell = True)
print(message)

b'--------------------------------------------------------------------------------\n                         | DOCUMENTATION OF COMMANDLINE CONTENTS |\n                         -----------------------------------------\nProgram:     plotGreenFKSpectra\nDescription: Plot Green frequency wavenumber spectra\n \nPositional arguments:\n                 dsvbase: Type =  sval, Value = gfk.MOMENT\n \nOptional arguments:\n           -jr: Type =  ival, Value   = 15\n           -bg: Type =  rval, Default = -4.0\n            -w: Type =  rval, Default = 8.\n            -a: Type =  rval, Default = 0.65\n--------------------------------------------------------------------------------\nTable of contents:\n  Name    Comp    Jump   Count\n     U       1       1       1\n     U       1       2       2\n     U       1       3       3\n     U       1       4       4\n     V       3       1       5\n     V       3       2       6\n     V       3       3       7\n     V       3       4       8\n     W       

You may look at individual seismograms using <b>plotSeismogramGather</b> or you may wish to collect event gathers and the plot these. To build event gathers, you may use the program <b>buildSeismogramGather</b> using again the synseis parameter file:

In [39]:
message = subprocess.check_output(binary_folder+"buildSeismogramGather "+parfile,shell = True)
print(message)

b"--------------------------------------------------------------------------------\n                         | DOCUMENTATION OF COMMANDLINE CONTENTS |\n                         -----------------------------------------\nProgram:     buildSeismogramGather\nDescription: Build up a seismic gather from individual synthetic traces\n \nPositional arguments:\n          gemini_parfile: Type =  sval, Value = parfile_syn\n \nOptional arguments:\n           -gt: Type =  sval, Default = SOURCE_GATHER\n--------------------------------------------------------------------------------\n The following values for the input parameters are used: \n 'PATH_SYNTHETIC_SEIS' = './synseis/'\n 'FILE_EVENT_LIST' = './events'\n 'FILE_STATION_LIST' = './stations'\n 'COMPONENTS' = 'ZNE'\n >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n >>>>>>>>>>>>>>>>   SUCCESS   >>>>>>>>>>>>>>>\n >>>> \n >>>> SUCCESS in readStandardEventFile --> successfully opened event file './events'\n >>>> SUCCESS in readStandardEventFile --> n

Now, we are ready to plot the event gathers:

In [40]:
with open(file_event_list,'r') as fd:
    csys = fd.readline()
    for line in fd:
        evid = (line.split(' '))[0]
        message = subprocess.check_output(binary_folder+"plotSeismogramGather "+path_synthetic_seis+evid+'_gather.asc',shell = True)
        print(message)

b'--------------------------------------------------------------------------------\n                         | DOCUMENTATION OF COMMANDLINE CONTENTS |\n                         -----------------------------------------\nProgram:     plotAsciiSynseisGather\nDescription: Plot a synthetic seismogram gather\n \nPositional arguments:\n            gather_files: Type =  sval, Value = ./synseis/060420_075254_gather.asc\n \nOptional arguments:\n           -tx: Type =  none, Default = none\n            -w: Type =  rval, Default = 10.\n            -a: Type =  rval, Default = 0.65\n     -pickfile: Type =  sval, Default = unspec.pck\n--------------------------------------------------------------------------------\n Reading gather file: ./synseis/060420_075254_gather.asc\n Available interactive options: A D X Z z l r u d\n Available interactive options: A D X Z z l r u d\n Available interactive options: A D X Z z l r u d\n Available interactive options: A D X Z z l r u d\n Available interactive opti